In [1]:
%store -r columns_encoded
%store -r columns
%store -r data
%store -r data_encoded

%store -r df_genres

columns_encoded = columns_encoded
columns = columns
data = data
data_encoded = data_encoded

df_genres = df_genres

In [2]:
#!python -m spacy download de_core_news_md

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import spacy

pd.set_option('display.max_columns', None)

In [5]:
text_data_lieblingssportart = data[columns.lieblingssportart]
text_data_lieblingsessen = data[columns.lieblingsessen]
text_data_lieblingsmusiker = data[columns.lieblingsmusiker]
text_data_lieblingsgenre = df_genres
text_data_combined = text_data_lieblingssportart + " " + text_data_lieblingsessen + " " + text_data_lieblingsmusiker + " " + df_genres

In [6]:
data_frame_content = {
    "lieblingssportart": text_data_lieblingssportart,
    "lieblingsessen": text_data_lieblingsessen,
    "lieblingsmusiker": text_data_lieblingsmusiker,
    "lieblingsgenre": text_data_lieblingsgenre,
    "combined": text_data_combined
}

text_data = pd.DataFrame(data_frame_content)

text_data.head(10)


,lieblingssportart,lieblingsessen,lieblingsmusiker,lieblingsgenre,combined
0,Joggen,Vegane Lasagne,Kraftklub,Rock,Joggen Vegane Lasagne Kraftklub Rock
1,Fußball,Butterbrot,Red Hot Chili Peppers,Rock,Fußball Butterbrot Red Hot Chili Peppers Rock
2,Keine Lieblingssportart,Lasagne,The 1975,Alternative,Keine Lieblingssportart Lasagne The 1975 Alter...
3,Trampolin,Burger,Kein Lieblingsmusiker,Kein Lieblingsgenre,Trampolin Burger Kein Lieblingsmusiker Kein Li...
4,Klettern,Dampfnudeln,Kein Lieblingsmusiker,Kein Lieblingsgenre,Klettern Dampfnudeln Kein Lieblingsmusiker Kei...
5,Bouldern,Pho,Kein Lieblingsmusiker,Kein Lieblingsgenre,Bouldern Pho Kein Lieblingsmusiker Kein Liebli...
6,Keine Lieblingssportart,Tomaten,Pugliese,Pop,Keine Lieblingssportart Tomaten Pugliese Pop
7,Fußball,Pasta,Green Day,Alternative,Fußball Pasta Green Day Alternative
8,Schnorcheln,Pizza,Pete Philly,Rap/Hip Hop,Schnorcheln Pizza Pete Philly Rap/Hip Hop
9,Tennis,Sushi,Radiohead,Alternative,Tennis Sushi Radiohead Alternative


# Wordvektoren

## Tfidf Vectoren

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
tfidf_lieblingssportart = TfidfVectorizer(max_df=0.25, min_df=1)
tfidf_vectors_lieblingssportart = tfidf_lieblingssportart.fit_transform(text_data.lieblingssportart)
tfidf_lieblingssportart.get_feature_names_out()

array(['badminton', 'basketball', 'bogenschießen', 'bouldern',
       'calisthenics', 'fahren', 'fitness', 'football', 'fußball',
       'hockey', 'inlineskaten', 'joggen', 'kampfsport', 'karate',
       'keine', 'klettern', 'kraftsport', 'lieblingssportart', 'padel',
       'radfahren', 'reiten', 'ruderboot', 'schnorcheln', 'schwimmen',
       'ski', 'sport', 'squash', 'tanzen', 'tauchen', 'tennis',
       'thaiboxen', 'tischtennis', 'trampolin', 'volleyball', 'yoga'],
      dtype=object)

In [9]:
tfidf_lieblingsessen = TfidfVectorizer(max_df=0.25, min_df=1)
tfidf_vectors_lieblingsessen = tfidf_lieblingsessen.fit_transform(text_data.lieblingsessen)
tfidf_lieblingsessen.get_feature_names_out()

array(['asiatische', 'bleu', 'bo', 'bratkartoffeln', 'burger', 'burrito',
       'butterbrot', 'carbonara', 'cordon', 'curry', 'dakgangjeong',
       'dampfnudeln', 'döner', 'gemüse', 'gemüsepfanne', 'gurke',
       'gurkensalat', 'humus', 'hähnchen', 'hühnchen', 'in', 'kein',
       'koreanisches', 'lachs', 'lasagne', 'lieblingsessen', 'mit',
       'mozzarella', 'nudeln', 'ofenkartoffeln', 'pasta', 'pfannkuchen',
       'pho', 'pilzen', 'pizza', 'pommes', 'ramen', 'reis', 'reispfanne',
       'risotto', 'sahnesauce', 'shakshuka', 'sojasoße', 'suppe', 'sushi',
       'tofu', 'tomaten', 'tortelini', 'und', 'vegane', 'viel', 'yaprak'],
      dtype=object)

In [10]:
tfidf_lieblingsmusiker = TfidfVectorizer(max_df=0.25, min_df=1)
tfidf_vectors_lieblingsmusiker = tfidf_lieblingsmusiker.fit_transform(text_data.lieblingsmusiker)
tfidf_lieblingsmusiker.get_feature_names_out()

array(['1975', '6lack', 'adele', 'and', 'antilopen', 'apache207',
       'arthur', 'avatar', 'avenged', 'bad', 'beethoven', 'billy',
       'chili', 'cicero', 'creator', 'day', 'deathspell', 'die',
       'dominic', 'down', 'drake', 'ed', 'eminem', 'fike', 'frusciante',
       'gang', 'gizzard', 'green', 'grimes', 'haftbefehl', 'halsey',
       'hans', 'harlow', 'hosen', 'hot', 'jack', 'jakey', 'james', 'john',
       'juice', 'kein', 'king', 'klangkuenstler', 'kraftklub', 'lapalux',
       'lieblingsmusiker', 'lizard', 'mac', 'malone', 'miller',
       'måneskin', 'neffex', 'nf', 'of', 'omega', 'omens', 'peppers',
       'pete', 'philly', 'post', 'prevail', 'provinz', 'pugliese',
       'radiohead', 'red', 'redlion', 'roger', 'sevenfold', 'sheeran',
       'sting', 'system', 'talent', 'the', 'toten', 'tyler', 'wizard',
       'wrld', 'zimmer'], dtype=object)

In [11]:
tfidf_lieblingsgenre = TfidfVectorizer(max_df=0.25, min_df=1)
tfidf_vectors_lieblingsgenre = tfidf_lieblingsgenre.fit_transform(text_data.lieblingsgenre)
tfidf_lieblingsgenre.get_feature_names_out()

array(['alternative', 'electro', 'filme', 'heavy', 'kein', 'klassik',
       'lieblingsgenre', 'metal', 'pop', 'rock', 'videospiele'],
      dtype=object)

## Spacy Wordvektoren

In [12]:
nlp = spacy.load("de_core_news_md")

In [13]:
tokens_lieblingsportart = []
for entry in text_data.lieblingssportart:
    tokens_lieblingsportart.append(nlp(entry))
text_data["tokens_lieblingssportart"] = tokens_lieblingsportart

In [14]:
tokens_lieblingsessen = []
for entry in text_data.lieblingsessen:
    tokens_lieblingsessen.append(nlp(entry))
text_data["tokens_lieblingsessen"] = tokens_lieblingsessen

In [15]:
tokens_lieblingsmusiker = []
for entry in text_data.lieblingsmusiker:
    tokens_lieblingsmusiker.append(nlp(entry))
text_data["tokens_lieblingsmusiker"] = tokens_lieblingsmusiker

In [16]:
tokens_lieblingsgenre = []
for entry in text_data.lieblingsgenre:
    tokens_lieblingsgenre.append(nlp(entry))
text_data["tokens_lieblingsgenre"] = tokens_lieblingsgenre

In [17]:
tokens_combined = []
for entry in text_data.combined:
    tokens_combined.append(nlp(entry))
text_data["tokens_combined"] = tokens_combined

In [18]:
text_data.head(10)

,lieblingssportart,lieblingsessen,lieblingsmusiker,lieblingsgenre,combined,tokens_lieblingssportart,tokens_lieblingsessen,tokens_lieblingsmusiker,tokens_lieblingsgenre,tokens_combined
0,Joggen,Vegane Lasagne,Kraftklub,Rock,Joggen Vegane Lasagne Kraftklub Rock,(Joggen),"(Vegane, Lasagne)",(Kraftklub),(Rock),"(Joggen, Vegane, Lasagne, Kraftklub, Rock)"
1,Fußball,Butterbrot,Red Hot Chili Peppers,Rock,Fußball Butterbrot Red Hot Chili Peppers Rock,(Fußball),(Butterbrot),"(Red, Hot, Chili, Peppers)",(Rock),"(Fußball, Butterbrot, Red, Hot, Chili, Peppers..."
2,Keine Lieblingssportart,Lasagne,The 1975,Alternative,Keine Lieblingssportart Lasagne The 1975 Alter...,"(Keine, Lieblingssportart)",(Lasagne),"(The, 1975)",(Alternative),"(Keine, Lieblingssportart, Lasagne, The, 1975,..."
3,Trampolin,Burger,Kein Lieblingsmusiker,Kein Lieblingsgenre,Trampolin Burger Kein Lieblingsmusiker Kein Li...,(Trampolin),(Burger),"(Kein, Lieblingsmusiker)","(Kein, Lieblingsgenre)","(Trampolin, Burger, Kein, Lieblingsmusiker, Ke..."
4,Klettern,Dampfnudeln,Kein Lieblingsmusiker,Kein Lieblingsgenre,Klettern Dampfnudeln Kein Lieblingsmusiker Kei...,(Klettern),(Dampfnudeln),"(Kein, Lieblingsmusiker)","(Kein, Lieblingsgenre)","(Klettern, Dampfnudeln, Kein, Lieblingsmusiker..."
5,Bouldern,Pho,Kein Lieblingsmusiker,Kein Lieblingsgenre,Bouldern Pho Kein Lieblingsmusiker Kein Liebli...,(Bouldern),(Pho),"(Kein, Lieblingsmusiker)","(Kein, Lieblingsgenre)","(Bouldern, Pho, Kein, Lieblingsmusiker, Kein, ..."
6,Keine Lieblingssportart,Tomaten,Pugliese,Pop,Keine Lieblingssportart Tomaten Pugliese Pop,"(Keine, Lieblingssportart)",(Tomaten),(Pugliese),(Pop),"(Keine, Lieblingssportart, Tomaten, Pugliese, ..."
7,Fußball,Pasta,Green Day,Alternative,Fußball Pasta Green Day Alternative,(Fußball),(Pasta),"(Green, Day)",(Alternative),"(Fußball, Pasta, Green, Day, Alternative)"
8,Schnorcheln,Pizza,Pete Philly,Rap/Hip Hop,Schnorcheln Pizza Pete Philly Rap/Hip Hop,(Schnorcheln),(Pizza),"(Pete, Philly)","(Rap, /, Hip, Hop)","(Schnorcheln, Pizza, Pete, Philly, Rap, /, Hip..."
9,Tennis,Sushi,Radiohead,Alternative,Tennis Sushi Radiohead Alternative,(Tennis),(Sushi),(Radiohead),(Alternative),"(Tennis, Sushi, Radiohead, Alternative)"


# Clustering mit Wortvektoren

In [19]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

In [20]:
tsne = TSNE()
lieblingssportart_vectors = np.array([token.vector for token in text_data.tokens_lieblingssportart])

tsne_vectors_lieblingssportart = tsne.fit_transform(lieblingssportart_vectors)

In [21]:
lieblingsessen_vectors = np.array([token.vector for token in text_data.tokens_lieblingsessen])
tsne_vectors_lieblingsessen = tsne.fit_transform(lieblingsessen_vectors)

In [22]:
lieblingsmusiker_vectors = np.array([token.vector for token in text_data.tokens_lieblingsmusiker])
tsne_vectors_lieblingsmusiker = tsne.fit_transform(lieblingsmusiker_vectors)

In [23]:
lieblingsgenre_vectors = np.array([token.vector for token in text_data.tokens_lieblingsgenre])
tsne_vectors_lieblingsgenre = tsne.fit_transform(lieblingsgenre_vectors)

In [24]:
tsne_vectors = { 
    'x_coord_lieblingssportart' : tsne_vectors_lieblingssportart[:, 0], 
    'y_coord_lieblingssportart' : tsne_vectors_lieblingssportart[:, 1],
    'x_coord_lieblingsessen' : tsne_vectors_lieblingsessen[:, 0], 
    'y_coord_lieblingsessen' : tsne_vectors_lieblingsessen[:, 1],
    'x_coord_lieblingsmusiker' : tsne_vectors_lieblingsmusiker[:, 0], 
    'y_coord_lieblingsmusiker' : tsne_vectors_lieblingsmusiker[:, 1],
    'x_coord_lieblingsgenre' : tsne_vectors_lieblingsgenre[:, 0], 
    'y_coord_lieblingsgenre' : tsne_vectors_lieblingsgenre[:, 1]
}

tsne_df = pd.DataFrame(tsne_vectors)

In [25]:
# n_cluster = Anzahl der Cluster
# distance_thershold = Distanz zwischen Punkte (nur wenn n_cluster = None)
ward = AgglomerativeClustering(linkage="ward",
                               n_clusters=4)

ward.fit(tsne_df)

# Cluster-Zentren
text_data["cluster"] = ward.labels_
text_data.head()

,lieblingssportart,lieblingsessen,lieblingsmusiker,lieblingsgenre,combined,tokens_lieblingssportart,tokens_lieblingsessen,tokens_lieblingsmusiker,tokens_lieblingsgenre,tokens_combined,cluster
0,Joggen,Vegane Lasagne,Kraftklub,Rock,Joggen Vegane Lasagne Kraftklub Rock,(Joggen),"(Vegane, Lasagne)",(Kraftklub),(Rock),"(Joggen, Vegane, Lasagne, Kraftklub, Rock)",2
1,Fußball,Butterbrot,Red Hot Chili Peppers,Rock,Fußball Butterbrot Red Hot Chili Peppers Rock,(Fußball),(Butterbrot),"(Red, Hot, Chili, Peppers)",(Rock),"(Fußball, Butterbrot, Red, Hot, Chili, Peppers...",3
2,Keine Lieblingssportart,Lasagne,The 1975,Alternative,Keine Lieblingssportart Lasagne The 1975 Alter...,"(Keine, Lieblingssportart)",(Lasagne),"(The, 1975)",(Alternative),"(Keine, Lieblingssportart, Lasagne, The, 1975,...",3
3,Trampolin,Burger,Kein Lieblingsmusiker,Kein Lieblingsgenre,Trampolin Burger Kein Lieblingsmusiker Kein Li...,(Trampolin),(Burger),"(Kein, Lieblingsmusiker)","(Kein, Lieblingsgenre)","(Trampolin, Burger, Kein, Lieblingsmusiker, Ke...",0
4,Klettern,Dampfnudeln,Kein Lieblingsmusiker,Kein Lieblingsgenre,Klettern Dampfnudeln Kein Lieblingsmusiker Kei...,(Klettern),(Dampfnudeln),"(Kein, Lieblingsmusiker)","(Kein, Lieblingsgenre)","(Klettern, Dampfnudeln, Kein, Lieblingsmusiker...",0


In [26]:
#!pip install plotly

In [27]:
combined_vectors = np.array([token.vector for token in text_data.tokens_combined])
tsne_vectors_combined = tsne.fit_transform(combined_vectors)
text_data["x_coord"] = tsne_vectors_combined[:, 0]
text_data["y_coord"] = tsne_vectors_combined[:, 1]

In [28]:
import plotly.express as px

df = px.data.iris()

features = df.loc[:, :'petal_width']

fig = px.scatter(
    text_data, x="x_coord", y="y_coord",
    color=text_data.cluster, labels={'color': 'species'},
    hover_data=["lieblingssportart", "lieblingsessen", "lieblingsmusiker", "lieblingsgenre"]
)
fig.show()